In [1]:
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import os
from os.path import isfile, join
import pandas as pd
from math import ceil
%matplotlib inline

Using TensorFlow backend.


In [2]:
master_image_size = 256, 256
master_color_channels = 1
random.seed(0)
base_image_dir = 'grayscale-256x256'

In [3]:
def get_new_model():
    """
    Creates a new CNN model. It also prints a summary of the
    model.
    
    It uses the external variables master_image_size and
    master_color_channels to setup the input layer.
    
    Returns
    -------
    keras.models.Sequential
        The model ready to be trained
    """
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(master_image_size[0], master_image_size[1], master_color_channels)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=0.5e-4), metrics=['acc'])
    
    model.summary()
    
    return model


In [4]:
def train_validation_test(base_path, shuffle_seed=0):
    """
    Reads images in a directory and splits them up according to class
    
    Assumes a binary classification, with EQUAL COUNTS in each
    class.
    
    This automatically assigns classes to the images. It assumes
    the name of each class is the first token of the filename as
    delimited by "_".
    
    The dataframes returned are to be used by Keras. They have the
    columns "filename" and "class" to point to the images and the
    classes, repectively.
    
    Parameters
    ----------
    base_path : str
        The relative path to the images.
        
    shuffle_seed : int
        The integer to seed the random number generator which shuffles
        the dataframe.
        
    Returns
    -------
    pd.DataFrame, pd.DataFrame, pd.DataFrame
        The train, validation and test sets, respectively.
    """
    print(f'>>> Shuffle seed {shuffle_seed}')
    
    train_fraction = 0.8
    random.seed(shuffle_seed)
    
    all_images_list = []
    
    for filename in os.listdir(base_path):
        if isfile(join(base_path, filename)):
            image_class = filename.split('.')[0]
            all_images_list.append({'class': image_class, 'filename': filename})
    
    all_images = pd.DataFrame(all_images_list)
    all_images = all_images.sample(frac=1).reset_index(drop=True)
    all_classes = all_images['class'].unique()
    
    first_class_name = all_classes[0]
    second_class_name = all_classes[1]
    
    first_class = all_images.copy().where(all_images['class'] == first_class_name).dropna()
    second_class = all_images.copy().where(all_images['class'] == second_class_name).dropna()

    train_row_count = int(len(first_class) * train_fraction)
    test_val_count = len(first_class) - train_row_count
    
    first_class_train = first_class.iloc[2 * test_val_count:]
    first_class_val = first_class.iloc[test_val_count:2 * test_val_count]
    first_class_test = first_class.iloc[0:test_val_count]
    
    second_class_train = second_class.iloc[2 * test_val_count:]
    second_class_val = second_class.iloc[test_val_count:2 * test_val_count]
    second_class_test = second_class.iloc[0:test_val_count]
    
    train = first_class_train.append(second_class_train).reset_index().drop('index', axis=1)
    val = first_class_val.append(second_class_val).reset_index().drop('index', axis=1)
    test = first_class_test.append(second_class_test).reset_index().drop('index', axis=1)
    
    print(first_class_name, second_class_name)
    
    return train, val, test

In [5]:
def train_validation_test_generators(src_dir, shuffle_seed=0):
    """
    Creates generators for train, validation and test datasets. These
    can then be used by Keras to train a model.
    
    Dataframe shuffling is prevented at this step because the dataframe
    is assumed to have been shuffled beforehand with an RNG with a known
    seed.
    
    The dataframe is created for you from the images in src_dir. See the
    train_validation_test function for more information.
    
    Parameters
    ----------
    src_dir : str
        The relative path to the train, validation and test images
        
    shuffle_seed : int
        The seed for the RNG used for dataframe shuffling.
        
    Returns
    -------
    """
    train, validation, test = train_validation_test(src_dir, shuffle_seed)

    train_datagen = image.ImageDataGenerator(rescale=1.0/255)
    test_datagen = image.ImageDataGenerator(rescale=1.0/255)
    validation_datagen = image.ImageDataGenerator(rescale=1.0/255)

    train_generator = train_datagen.flow_from_dataframe(dataframe=train,
                                                        directory=src_dir,
                                                        target_size=master_image_size,
                                                        batch_size=20,
                                                        shuffle=False,
                                                        color_mode='grayscale',
                                                        class_mode='binary')

    validation_generator = train_datagen.flow_from_dataframe(dataframe=validation,
                                                      directory=src_dir,
                                                      target_size=master_image_size,
                                                      batch_size=20,
                                                      shuffle=False,
                                                      color_mode='grayscale',
                                                      class_mode='binary')

    test_generator = train_datagen.flow_from_dataframe(dataframe=test,
                                                       directory=src_dir,
                                                       target_size=master_image_size,
                                                       batch_size=20,
                                                       shuffle=False,
                                                       color_mode='grayscale',
                                                       class_mode='binary')
    
    return train_generator, validation_generator, test_generator

In [6]:
def train_model_and_get_history(base_image_dir, shuffle_seed=0, epochs=10):
    """
    Trains a model and returns the RNG seed and history
    
    Parameters
    ----------
    base_image_dir : str
        The relative path to the images.
        
    shuffle_seed : int
        The seed for the RNG that shuffles the train, validation
        and test datasets.
        
    Returns
    -------
    keras.model.Sequential, dict
        The model that created the history
    """
    train_generator, validation_generator, test_generator = train_validation_test_generators(base_image_dir, shuffle_seed)
    model = get_new_model()
    train_history = model.fit_generator(train_generator,
                                        steps_per_epoch=100,
                                        epochs=epochs,
                                        validation_data=validation_generator,
                                        validation_steps=50,
                                        verbose=1)
    return model, train_history

In [7]:
def train_and_run_different_shuffles(shuffles, src_image_dir, epochs_per_shuffle=10):
    histories = []
    for seed in range(shuffles):
        model, history = train_model_and_get_history(src_image_dir, seed, epochs=epochs_per_shuffle)
        histories.append({
            'seed': seed,
            'epochs': epochs_per_shuffle,
            'history': history,
            'model': model
        })
    return histories

In [8]:
histories = train_and_run_different_shuffles(10, src_image_dir=base_image_dir, epochs_per_shuffle=10)

>>> Shuffle seed 0
benzene_ring non_benzene_ring
Found 246 validated image filenames belonging to 2 classes.
Found 82 validated image filenames belonging to 2 classes.
Found 82 validated image filenames belonging to 2 classes.
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 96)      55392     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 6

ResourceExhaustedError: OOM when allocating tensor with shape[20,64,254,254] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_5/RMSprop/gradients/max_pooling2d_21/MaxPool_grad/MaxPoolGrad}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
